In [1]:
import cv2
import mediapipe as mp

In [2]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False,
                                   max_num_faces=1,
                                   refine_landmarks=True,
                                   min_detection_confidence=0.5,
                                   min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils

In [5]:
from math import dist
def euclidean_distance(point1, point2):
    return dist(point1, point2)

In [7]:
def calculate_ear(landmarks, eye_indices):
    left = landmarks[eye_indices[0]]
    right = landmarks[eye_indices[3]]

    top_center = (
        (landmarks[eye_indices[1]][0] + landmarks[eye_indices[2]][0]) / 2,
        (landmarks[eye_indices[1]][1] + landmarks[eye_indices[2]][1]) / 2
    )
    bottom_center = (
        (landmarks[eye_indices[4]][0] + landmarks[eye_indices[5]][0]) / 2,
        (landmarks[eye_indices[4]][1] + landmarks[eye_indices[5]][1]) / 2
    )

    horizontal_length = euclidean_distance(left, right)
    vertical_length = euclidean_distance(top_center, bottom_center)

    ear = vertical_length / horizontal_length
    return ear

In [8]:
LEFT_EYE_INDICES = [33, 160, 158, 133, 153, 144]  
RIGHT_EYE_INDICES = [362, 385, 387, 263, 373, 380]

In [10]:
UPPER_LIP_TOP = 13
LOWER_LIP_BOTTOM = 14

In [11]:
def calculate_mouth_opening(landmarks):
    top = landmarks[UPPER_LIP_TOP]
    bottom = landmarks[LOWER_LIP_BOTTOM]
    return euclidean_distance(top, bottom)

In [ ]:
import cv2
import threading
from playsound import playsound
def play_alert():
    threading.Thread(target=playsound, args=("alert.mp3",), daemon=True).start()

EAR_THRESHOLD = 0.25
EAR_CONSEC_FRAMES = 20
MOUTH_OPEN_THRESHOLD = 25
YAWN_CONSEC_FRAMES = 15

counter = 0
yawn_counter = 0

cap = cv2.VideoCapture(0)

while True:
    success, frame = cap.read()
    if not success:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            h, w, _ = frame.shape
            landmarks = [(int(pt.x * w), int(pt.y * h)) for pt in face_landmarks.landmark]

            # === EAR Calculation ===
            left_ear = calculate_ear(landmarks, LEFT_EYE_INDICES)
            right_ear = calculate_ear(landmarks, RIGHT_EYE_INDICES)
            avg_ear = (left_ear + right_ear) / 2

            if avg_ear < EAR_THRESHOLD:
                counter += 1
                if counter == EAR_CONSEC_FRAMES:
                    cv2.putText(frame, "DROWSINESS ALERT!", (30, 100),
                                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
                    play_alert()
            else:
                counter = 0

            mouth_opening = euclidean_distance(landmarks[13], landmarks[14])
            if mouth_opening > MOUTH_OPEN_THRESHOLD:
                yawn_counter += 1
                if yawn_counter == YAWN_CONSEC_FRAMES:
                    cv2.putText(frame, "YAWNING ALERT!", (30, 140),
                                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 3)
                    play_alert()
            else:
                yawn_counter = 0

            cv2.putText(frame, f'EAR: {avg_ear:.2f}', (30, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            cv2.putText(frame, f'Mouth: {mouth_opening:.2f}', (30, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

    cv2.imshow("Drowsiness & Yawning Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()